In [4]:
%pip install openAI
%pip install langchain
%pip install typing-inspect==0.8.0 typing_extensions==4.5.0
%pip install pydantic==1.10.8

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import openai
from langchain import *
from pydantic import *
import os 
import pandas as pd 
with open("secret.txt") as f:
    key=f.read()
os.environ['OPENAI_API_KEY']=key[:-1] #for some reason there's a new line, so we remove it



# Let's try LangChain

Literally just grabbed the example from the website and modified. still have no idea how this works but whatever (for now) 


## Putting everything together

In [8]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List



# Define your desired data structure.
class extracted_data(BaseModel):
    descriptions: list = Field(description="A list of the descriptive phrases from the transcript. This should be a list of strings.")
    
    





input =pd.read_csv("data/test/sample.csv")

parser = PydanticOutputParser(pydantic_object=extracted_data)

info_query_generic = "Here's a partial transcript of people describing images. Extract verbatim a list of the descriptive phrases that are used. \
If there are no descriptive phrases used, return an empty list. \n\n \
  As an example, if the transcript was 'It looks like a magician, and, uhh, I think he's got a rabbit.', the response would be ['a magician','he's got a rabbit'].\n\n \
     As an example, if the transcript was 'big triangle arm facing left head on the right', the response would be ['big triangle arm', 'facing left', 'head on the right'].\n\n \
      Return just a list of the descriptive phrases. Here's the text:\n\n "
model_name = "gpt-3.5-turbo"
#model_name="gpt-4"
temperature = 0
model = ChatOpenAI(temperature=temperature)


def code_data_with_langchain(df, model, parser, info_query_generic): 
  langchain_prompt = PromptTemplate(
    template="{query}\n",
    input_variables=["query"]
  )
  system_prompt = "You are a competent researcher who answers user queries."

  res = []
  #for index in range(100):
  for index, _ in df.iterrows():
    print(index)
    info_query = info_query_generic + df["text"][index] + "\n\n"
    _input = langchain_prompt.format_prompt(query=info_query)
    #print(_input)
    full_prompt=[SystemMessage(content=system_prompt), HumanMessage(content=_input.to_string())]
    #print(full_prompt)
    output = model(full_prompt)
    #parsed_output = parser.parse(output)
    string_output=output.content
    print(string_output)
    res.append(string_output)
  return res


res = code_data_with_langchain(input, model, parser,  info_query_generic)

output=input
output['gpt_out']=res
output.to_csv("data/test/outsample_gpt3.csv")





0
['facing left', 'head to the right', 'over his back', 'leg out to the left']
1
['his arms are level with his head']
2
['his head comes past the right side of his body by a bit']
3
['the only one with a leg out to the left']
4
['the persons legs are facing the left', 'his head which is shaped almost like the box is facing to the right']
5
['walking to the left']
6
['a bit like the thriller dance']
7
['a man walking to the left', 'with a square backpack on', 'triangle on his face']
8
['one leg on floor to the left', 'one leg left is slightly raised']
9
['leg lifted to the left']
10
['thriller dance']
11
[]
12
['thriller']
13
['a person dancing', 'one foot on the floor', 'another foot is raised']
14
['tilted back']
15
['made from a square shape']
16
['4 sided shape']
17
['the last one', 'we haven't selected']
18
['the one with 2 heads almost']
19
['dancing man', '1 foot up', '1 down', '2 heads']
20
['dancing man']
21
['dancing man']
22
['two heads']
23
['dancing man']
24
['dancing man']